In [1]:
using CommonDataFormat
using Dates
using Statistics

In [ ]:
omni_file = joinpath(pwd(), "../Data/MAVEN/mvn_swe_l2_arc3d_20210824_v05_r01.cdf")
ds = CDFDataset(omni_file)

LoadError: SystemError: opening file "e:\\WorkSpace\\Tianwen\\src\\../../Data/MAVEN/mvn_swe_l2_arc3d_20210824_v05_r01.cdf": No such file or directory

In [4]:
epoch = ds["epoch"][:]
energy = ds["energy"][:]
#flux_3d = ds["counts"][:,:,:,:]

64-element Vector{Float32}:
 4627.5
 4118.506
 3665.4976
 3262.3171
 2903.484
 2584.12
 2299.884
 2046.9121
 1821.7654
 1621.3833
 1443.0419
 1284.3169
 1143.0505
    ⋮
   10.809248
    9.620303
    8.562134
    7.6203556
    6.7821674
    6.036174
    5.372235
    4.781325
    4.2554107
    3.787344
    3.3707614
    3.0

In [5]:
using CairoMakie
using LinearAlgebra
using Interpolations

target_time_str = "2021-08-24T07:30:00"
# 将目标时间字符串解析为 DateTime 对象
target_dt = DateTime(target_time_str, dateformat"yyyy-mm-ddTHH:MM:SS")

epoch = ds["epoch"][:]
energy = ds["energy"][:]
flux_4d = ds["diff_en_fluxes"][:, :, :, :]
times_dates = DateTime.(epoch)
time_val, time_idx = findmin(d -> abs(d - target_dt), times_dates)

output_file = joinpath(pwd(), "../Picture/MAVEN/MAVEN_e_spectrum$(epoch[time_idx]).png")

# 选择一个探测器（这里选择第一个探测器，dindex=0）
detector_idx = 1  # 对应第一个探测器
# 提取该时间点、该探测器的数据
# 注意：flux_4d 形状为 (6, 16, 64, 10800)
# 我们取第一个探测器，所有方位角，所有能量通道，指定时间点
flux_3d = flux_4d[detector_idx, :, :, time_idx]  # 形状: (16 角度索引, 64 能量通道)

# 扫描角度：根据图中标注，扫描角度为 0° 到 360°
# CDF 中没有直接给出扫描角度数组，但可以从角度索引 0-15 对应 0-360° 均匀分布
num_angles = size(flux_3d, 1)  # 16
scan_angle = LinRange(0, 360, num_angles)  # 0° 到 360°

# 选择要绘制的能量值（对应图中标注的能量：4, 10, 20, 35, 57, 92, 144, 222 eV）
target_energies = [4, 10, 20, 35, 57, 92, 144, 222]

# 找到最接近目标能量的索引
energy_indices = [argmin(abs.(energy .- E)) for E in target_energies]

# 设置颜色循环
colors = [
    :blue, :green, :red, :cyan,
    :magenta, :orange, :brown, :purple
]

# 创建 CairoMakie 图形
fig = Figure(resolution=(800, 600))
ax = Axis(fig[1, 1],
    xlabel="SCAN ANGLE (deg)",
    ylabel="DIFFERENTIAL ENERGY FLUX\n(eV/[eV cm² sr s])",
    xticks=0:90:360,
    yscale=log10,
    title="ELECTRON DISTRIBUTION MODEL 2\nTime: $(epoch[time_idx])"
)

# 设置坐标轴限制
xlims!(ax, 0, 360)

# 对每个目标能量绘制曲线
for (i, (E, idx, col)) in enumerate(zip(target_energies, energy_indices, colors))
    flux_at_E = flux_2d[:, idx]  # 在当前能量下，随扫描角度的通量
    # 处理无效值（如填充值 -1e31）
    valid = flux_at_E .> 0
    if any(valid)
        lines!(ax, scan_angle[valid], flux_at_E[valid],
            color=col, linewidth=1.5,
            label="$(E) eV")
    end
end

# 添加图例
axislegend(ax, position=:rt, title="Energy (eV)", framevisible=false)

# 保存图像（可选）
save(output_file, fig)

┌ Warning: Found `resolution` in the theme when creating a `Scene`. The `resolution` keyword for `Scene`s and `Figure`s has been deprecated. Use `Figure(; size = ...` or `Scene(; size = ...)` instead, which better reflects that this is a unitless size and not a pixel resolution. The key could also come from `set_theme!` calls or related theming functions.
└ @ Makie C:\Users\USTC\.julia\packages\Makie\Vn16E\src\scenes.jl:264


LoadError: UndefVarError: `flux_2d` not defined

In [ ]:
times = ds["epoch"][:]
fieldnames(typeof(times[1]))

In [57]:
ds["epoch"][:]

26926-element Vector{TT2000}:
 2021-08-24T00:57:40.078
 2021-08-24T00:57:42.078
 2021-08-24T00:57:44.078
 2021-08-24T00:57:46.078
 2021-08-24T00:57:48.078
 2021-08-24T00:57:50.078
 2021-08-24T00:57:52.078
 2021-08-24T00:57:54.078
 2021-08-24T00:57:56.078
 2021-08-24T00:57:58.078
 2021-08-24T00:58:00.078
 2021-08-24T00:58:02.078
 2021-08-24T00:58:04.077
 ⋮
 2021-08-24T23:59:37.421
 2021-08-24T23:59:39.421
 2021-08-24T23:59:41.421
 2021-08-24T23:59:43.421
 2021-08-24T23:59:45.421
 2021-08-24T23:59:47.421
 2021-08-24T23:59:49.421
 2021-08-24T23:59:51.421
 2021-08-24T23:59:53.421
 2021-08-24T23:59:55.421
 2021-08-24T23:59:57.421
 2021-08-24T23:59:59.421

In [43]:
ds["diff_en_fluxes"][:, :, :]

16×64×26926 Array{Float32, 3}:
[:, :, 1] =
 NaN    NaN    NaN    NaN    NaN    NaN    NaN    …  NaN          NaN
 NaN    NaN    NaN    NaN    NaN    NaN    NaN       NaN          NaN
 NaN    NaN    NaN    NaN    NaN    NaN    NaN       NaN          NaN
   0.0    0.0    0.0    0.0    0.0    0.0    0.0       3.66145f7    5.10245f7
   0.0    0.0    0.0    0.0    0.0    0.0    0.0       4.6327f7     7.07571f7
   0.0    0.0    0.0    0.0    0.0    0.0    0.0  …    7.00063f7    1.02697f8
   0.0    0.0    0.0    0.0    0.0    0.0    0.0       7.24895f7    3.92311f7
   0.0    0.0    0.0    0.0    0.0    0.0    0.0       7.95061f7    6.31518f7
   0.0    0.0    0.0    0.0    0.0    0.0    0.0       3.79254f7    3.77162f7
   0.0    0.0    0.0    0.0    0.0    0.0    0.0       3.31365f7    5.71987f7
   0.0    0.0    0.0    0.0    0.0    0.0    0.0  …    7.61884f7    7.11069f7
   0.0    0.0    0.0    0.0    0.0    0.0    0.0       1.61458f8    1.56238f8
   0.0    0.0    0.0    0.0    0.0    0.0    

In [41]:
ds["energy"][:]

64-element Vector{Float32}:
 4627.5
 4118.506
 3665.4976
 3262.3171
 2903.484
 2584.12
 2299.884
 2046.9121
 1821.7654
 1621.3833
 1443.0419
 1284.3169
 1143.0505
    ⋮
   10.809248
    9.620303
    8.562134
    7.6203556
    6.7821674
    6.036174
    5.372235
    4.781325
    4.2554107
    3.787344
    3.3707614
    3.0

In [ ]:
ds["pa"][:]